In [3]:
import subprocess
import xarray as xr
import numpy as np
import rasterio
from herbie import Herbie

In [5]:
# file = 'sample/14112023.nc'
# ds = xr.open_dataset(file)

H = Herbie(
  '2024-06-05',
  model='ifs',
  product='oper',
  fxx=0
)
search = ":[u|v]:1000:"


✅ Found ┊ model=ifs ┊ product=oper ┊ 2024-Jun-05 00:00 UTC F00 ┊ GRIB2 @ azure ┊ IDX @ azure


In [6]:
H.download(search)

👨🏻‍🏭 Created directory: [C:\Users\falbas\data\ifs\20240605]


WindowsPath('C:/Users/falbas/data/ifs/20240605/subset_5defb960__20240605000000-0h-oper-fc.grib2')

In [7]:
ds = H.xarray(search)

c:\Users\falbas\miniconda3\Lib\site-packages\herbie\core.py:1097: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")


In [8]:
ds

<xarray.Dataset> Size: 8MB
Dimensions:              (latitude: 721, longitude: 1440)
Coordinates:
    time                 datetime64[ns] 8B 2024-06-05
    step                 timedelta64[ns] 8B 00:00:00
    isobaricInhPa        float64 8B 1e+03
  * latitude             (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude            (longitude) float64 12kB -180.0 -179.8 ... 179.5 179.8
    valid_time           datetime64[ns] 8B 2024-06-05
    gribfile_projection  object 8B None
Data variables:
    u                    (latitude, longitude) float32 4MB ...
    v                    (latitude, longitude) float32 4MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    model:                   ifs
    product:                 oper
    description:             ECMWF Open Data - Integrated Forecast System
    remote_grib:             https://ai4edataeuwest.blob.core.windows.net/ecm...
    local_grib:              C:\Users\falbas\data\ifs\20240605\subset_5defb96...
    search:                  :[u|v]:1000:

In [16]:
lat = ds["latitude"][280:441].values
lon = ds["longitude"][1040:1361].values
u = ds["u"][280:441, 1040:1361].values
v = ds["v"][280:441, 1040:1361].values

In [10]:
ws = np.sqrt(u**2 + v**2)

In [17]:
transform = rasterio.transform.from_bounds(
    west=lon[0],
    south=lat[-1],
    east=lon[-1],
    north=lat[0],
    width=lon.shape[0],
    height=lat.shape[0],
)

In [19]:
# ws = np.flip(ws, axis=0)
with rasterio.open(
    "output/v.tif",
    "w",
    driver="GTiff",
    height=lat.shape[0],
    width=lon.shape[0],
    count=1,
    dtype=u.dtype,
    crs="+proj=latlong",
    transform=transform,
    nodata=-9999,
) as dst:
    dst.write(v, 1)

In [15]:
subprocess.run(
    [
        "gdaldem",
        "color-relief",
        "-of",
        "JPEG",
        "output/wspd.tif",
        "color/wspd_color.txt",
        "output/wspd.jpg",
    ]
)

CompletedProcess(args=['gdaldem', 'color-relief', '-of', 'JPEG', 'output/wspd.tif', 'color/wspd_color.txt', 'output/wspd.jpg'], returncode=0)